In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from IPython.display import clear_output
import pandas as pd
import random

In [67]:
df_urls = pd.read_csv('urls_propiedades_en_venta_san_martin_argenprop', index_col=False)
urls = df_urls['0'].tolist()
urls

['https://www.argenprop.com/departamento-en-venta-en-villa-ballester-2-ambientes--11588507',
 'https://www.argenprop.com/departamento-en-venta-en-villa-ballester-2-ambientes--11352489',
 'https://www.argenprop.com/departamento-en-venta-en-villa-ballester-2-ambientes--11424033',
 'https://www.argenprop.com/departamento-en-venta-en-villa-ballester-2-ambientes--11382103',
 'https://www.argenprop.com/departamento-en-venta-en-villa-ballester-3-ambientes--10574853',
 'https://www.argenprop.com/departamento-en-venta-en-villa-ballester-2-ambientes--11599268',
 'https://www.argenprop.com/departamento-en-venta-en-villa-ballester-4-ambientes--7847590',
 'https://www.argenprop.com/departamento-en-venta-en-villa-ballester-3-ambientes--11592904',
 'https://www.argenprop.com/departamento-en-venta-en-villa-ballester-3-ambientes--11468838',
 'https://www.argenprop.com/departamento-en-venta-en-centro-2-ambientes--11547948',
 'https://www.argenprop.com/departamento-en-venta-en-centro-3-ambientes--1159140

In [42]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.argenprop.com/departamento-en-venta-en-general-san-martin-3-ambientes--8471698')
element = driver.find_element(By.CLASS_NAME, 'map-container')
element = element.find_element(By.TAG_NAME, 'div')

[WDM] - ====== WebDriver manager ======


2022-07-10 19:30:26,588 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 102.0.5005


2022-07-10 19:30:26,683 INFO Current google-chrome version is 102.0.5005


[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome


2022-07-10 19:30:26,687 INFO Get LATEST chromedriver version for 102.0.5005 google-chrome


[WDM] - Driver [/home/nvrancovich/.wdm/drivers/chromedriver/linux64/102.0.5005.61/chromedriver] found in cache


2022-07-10 19:30:26,812 INFO Driver [/home/nvrancovich/.wdm/drivers/chromedriver/linux64/102.0.5005.61/chromedriver] found in cache


In [82]:
%%time

features_global = []
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for url in random.sample(urls, 50):
    features_local = {}
    driver.get(url)
    
    # Extraccion del precio en USD
    
    element = driver.find_element(By.CLASS_NAME, 'titlebar__price').text
    element = element.replace('USD', '')
    element = element.replace('.', '')
    element = element.replace(' ', '')
    features_local['precio_USD'] = int(element)
    
    # Categoria de la propiedad (casa, departamento o ph)
    
    if 'casa' in url:
        features_local['categoria'] = 'casa'
    elif 'ph' in url: 
        features_local['categoria'] = 'ph'
    elif 'departamento' in url:
        features_local['categoria'] = 'departamento'
    
    # Extraccion de los elementos principales
    
    try:
        element = driver.find_element(By.CLASS_NAME, 'property-main-features')
        elements = element.find_elements(By.TAG_NAME, 'p')
    except:
        pass
    
    try:
        features_local[elements[0].text] = elements[1].text
    except:
        pass

    try:
        features_local[elements[2].text] = elements[3].text
    except:
        pass

    try:
        features_local[elements[4].text] = elements[5].text
    except:
        pass

    try:
        features_local[elements[6].text] = elements[7].text
    except:
        pass

    try:
        features_local[elements[8].text] = elements[9].text
    except:
        pass

    # Correcion de elementos que son equivalentes, son inecesarios o hay que formatear.
     
    try:
        features_local['Baños'] = features_local['Cantidad de baños']
        del features_local['Cantidad de baños']
    except:
        pass
    
    try:
        features_local['Dormitorios'] = features_local['Cantidad de dormitorios']
        del features_local['Cantidad de dormitorios']
    except:
        pass
    
    try:
        features_local['Antiguedad'] = features_local['Antigüedad']
        del features_local['Antigüedad']
    except:
        pass
    
    try:
        features_local['Superficie cubierta'] = features_local['Superficie cubierta'].replace('m²', '')
        features_local['Superficie cubierta'] = features_local['Superficie cubierta'].replace(' ', '')
        features_local['Superficie cubierta'] = int(features_local['Superficie cubierta'])
    except:
        pass
    
    try:
        del features_local['Disposición']
    except:
        pass
    
    # Extracion de la latitud y la longitud
    
    try:
        element = driver.find_element(By.CLASS_NAME, 'map-container')
        element = element.find_element(By.TAG_NAME, 'div').get_attribute('data-latitude')
        features_local['latitud'] = float(element.replace(',','.'))
    except:
        pass
    
    try:
        element = driver.find_element(By.CLASS_NAME, 'map-container')
        element = element.find_element(By.TAG_NAME, 'div').get_attribute('data-longitude')
        features_local['logitud'] = float(element.replace(',','.'))
    except:
        pass
    
    
    features_global.append(features_local)
    clear_output(wait=True)
    print(f'Propiedad número {urls.index(url) + 1} extraída ({int(len(features_local))} elementos)')

driver.close()
    
df = pd.DataFrame(features_global)

Propiedad número 5839 extraída (7 elementos)
CPU times: user 3.09 s, sys: 279 ms, total: 3.37 s
Wall time: 5min 7s


In [83]:
df

,precio_USD,categoria,Superficie cubierta,Dormitorios,Antiguedad,latitud,logitud,Baños
0,60000,ph,57.0,2,20 años,-34.554947,-58.539272,NaN
1,220000,casa,121.0,3,15 años,-34.559753,-58.546850,2
2,50000,ph,42.0,1,50 años,-34.554478,-58.573620,1
3,85000,ph,90.0,3,A Estrenar,-34.553430,-58.566914,1
4,75000,departamento,42.0,1,A Estrenar,-34.574844,-58.536530,1
5,65000,ph,70.0,2,NaN,-34.586790,-58.545960,1
6,250000,casa,220.0,4,6 años,-34.562828,-58.540382,2
7,180000,departamento,118.0,3,40 años,-34.575220,-58.534763,2
8,89900,casa,150.0,2,40 años,-34.521450,-58.574615,1
9,85000,casa,87.0,2,40 años,-34.533394,-58.568596,1


In [85]:
df.groupby('categoria').describe()[['precio_USD', 'Superficie cubierta']]

precio_USD                                                 \
                  count           mean           std      min      25%   
categoria                                                                
casa               21.0  162804.761905  83369.001890  40000.0  92000.0   
departamento       15.0  121666.666667  65228.900986  39000.0  71500.0   
ph                 14.0   70921.428571  24265.080278  35000.0  52000.0   

                                           Superficie cubierta              \
                   50%       75%       max               count        mean   
categoria                                                                    
casa          130000.0  249000.0  360000.0                19.0  133.315789   
departamento   95000.0  170000.0  240000.0                15.0   66.933333   
ph             65000.0   91675.0  119000.0                14.0   70.357143   

                                                           
                    std   min   25%    50%     75%    max  
categoria                                                  
casa          53.392730  26.0  99.5  140.0  150.00  241.0  
departamento  38.958708  28.0  41.0   60.0   70.00  177.0  
ph            38.557863  30.0  44.0   55.5   88.25  150.0